In [1]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB


%matplotlib inline

In [2]:
#Bring in the data

nba = pd.read_csv('./data/nba_analysis_data.csv')
nba.dropna(inplace = True)

In [3]:
 features = ['home_win_pct', 'away_win_pct',
             'eff_ratio1','eff_ratio2',
            'mov_5_fta', 'mov_5_away_fta',  
             'eff_ratio3', 'eff_ratio4', 
            'mov_5_home_score', 'mov_5_away_score',
            'mov_5_away_off_eff', 'mov_5_away_def_eff', 
            'mov_5_away_assists', 'mov_5_home_win_margin',
            'mov_5_win', 'home_ave_win_margin', 
            'mov_5_away_win_margin', 'home_win_pct', 'away_win_pct',
            'high_alt', 'home_ave_win_margin', 'away_ave_win_margin',
            'playoff_game', 'mov_5_3pa', 'mov_5_away_3pa',
            'mov_2_fta', 'mov_2_away_fta', 
           'mov_2_home_score', 'mov_2_away_score',
           'mov_2_tot', 'mov_2_away_total_reb',
           'mov_2_away_off_eff', 'mov_2_away_def_eff', 
            'mov_2_away_assists', 'mov_2_home_win_margin',
           'mov_2_win', 'mov_2_away_win_margin', 'mov_2_3pa', 
            'mov_2_away_3pa','away_rest', 'rest_days'
            ]
X = nba.drop(columns = ['cover', 'home_win_margin'])

y = nba['cover']

In [4]:
#splitting data for a train-test split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y ,
                                                    stratify=y,
                                                    random_state = 42)

In [5]:
#scaling data to use in various other methods
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [6]:
lr = LogisticRegressionCV(penalty='l1', solver ='saga', cv =3 )
lr.fit(X_train_scaled, y_train)
lr.score(X_train_scaled, y_train)

/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.5300131444428217

In [7]:
lr.score(X_test_scaled, y_test)

0.5212439772229522

In [8]:
y_hat_lr = lr.predict(X_test_scaled)
y_hat_lrp = lr.predict_proba(X_test_scaled)

In [9]:
print(classification_report(y_test, y_hat_lr))

              precision    recall  f1-score   support

           0       0.53      0.83      0.64      1196
           1       0.49      0.18      0.27      1087

   micro avg       0.52      0.52      0.52      2283
   macro avg       0.51      0.51      0.46      2283
weighted avg       0.51      0.52      0.46      2283



In [10]:
confusion_matrix( y_test, y_hat_lr)

array([[991, 205],
       [888, 199]])

### Naive Bayes

Model seems to work best with bare bones information.  More information confused the model.

In [11]:
nb = GaussianNB(var_smoothing=4)
nb.fit(X_train_scaled, y_train)
print(nb.score(X_train_scaled, y_train))
print(nb.score(X_test_scaled, y_test))

0.5282605520665985
0.5260621988611476


In [12]:
y_hat_nb = nb.predict(X_test_scaled)

In [13]:
print(classification_report(y_test, y_hat_nb))

              precision    recall  f1-score   support

           0       0.53      0.95      0.68      1196
           1       0.52      0.06      0.11      1087

   micro avg       0.53      0.53      0.53      2283
   macro avg       0.52      0.50      0.39      2283
weighted avg       0.52      0.53      0.41      2283



In [14]:
confusion_matrix( y_test, y_hat_nb)

array([[1133,   63],
       [1019,   68]])